In [1]:
import csv
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import helpers as analysis
from numpy import linalg as LA
import numpy as np

[nltk_data] Downloading package punkt to /Users/Home/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df = pd.read_csv('backend/data/coffee_fix.csv')
df

In [45]:
combined_descriptions = df[["desc_1"]].apply(lambda x : ' '.join(x.dropna()), axis=1)
combined_names = df[["name"]].apply(lambda x : ' '.join(x.dropna()), axis=1)
combined_locs = df[["origin"]].apply(lambda x : ' '.join(x.dropna()), axis=1)

combined_descriptions = [x for x in combined_descriptions]
combined_names = [x + " from " + combined_locs[i] for i, x in enumerate(combined_names)]

vectorizer = TfidfVectorizer()
doc_by_vocab  = vectorizer.fit_transform(combined_descriptions).toarray()

index_to_vocab = {i:v for i, v in enumerate(vectorizer.get_feature_names_out())}
vocab_to_index = {index_to_vocab[i]:i for i in index_to_vocab}
print(vocab_to_index)
doc_to_index = {v:i for i, v in enumerate(combined_names)}

query = "Nutty and Sweet"
query = query.lower()
query = query.split(" ")

queryVector = [0] * doc_by_vocab.shape[1]

for c in query: 
  if c in vocab_to_index:
    queryVector[vocab_to_index[c]] +=1 


similarities = []
for i in range(len(doc_by_vocab)): 
  similarities.append((np.dot(queryVector, doc_by_vocab[i]) / (LA.norm(queryVector) * LA.norm(doc_by_vocab[i])), i))

similarities.sort(reverse = True)
topTen = similarities[:10]

Names = [combined_names[i] for x, i in topTen]
print(Names)




{'250': 0, '25ml': 1, '85': 2, 'about': 3, 'accessible': 4, 'accessibly': 5, 'accompanied': 6, 'acdity': 7, 'acid': 8, 'acidity': 9, 'acidty': 10, 'acidy': 11, 'acorn': 12, 'acrid': 13, 'add': 14, 'added': 15, 'adding': 16, 'addition': 17, 'adds': 18, 'adult': 19, 'after': 20, 'agave': 21, 'aged': 22, 'agreeably': 23, 'akin': 24, 'alcohol': 25, 'alive': 26, 'all': 27, 'allspice': 28, 'almond': 29, 'almonds': 30, 'almondy': 31, 'almost': 32, 'aloe': 33, 'aloewood': 34, 'along': 35, 'alongside': 36, 'already': 37, 'also': 38, 'alyssum': 39, 'amber': 40, 'amond': 41, 'among': 42, 'amplification': 43, 'amplified': 44, 'amplifies': 45, 'amplify': 46, 'amplifying': 47, 'amps': 48, 'an': 49, 'anchor': 50, 'anchored': 51, 'and': 52, 'animate': 53, 'animated': 54, 'animates': 55, 'animating': 56, 'anise': 57, 'another': 58, 'appealing': 59, 'appealingly': 60, 'appearances': 61, 'apple': 62, 'apricot': 63, 'aprium': 64, 'are': 65, 'aroma': 66, 'aromas': 67, 'aromatic': 68, 'aromatically': 69, 'a